# Environment Setup

In [13]:
%pip install unidecode
%pip install pandas
%pip install functools

import pandas as pd
from unidecode import unidecode
from functools import reduce
import re

# Check if running in Google Colab, to download the datasets
import sys
if 'google.colab' in sys.modules:
    print("Running in Google Colab. Downloading datasets...")
    !git init
    !git remote add origin https://github.com/FrostPrice/kdd-queimadas-nas-ucs.git
    !git config core.sparseCheckout true
    !echo "datasets/" >> .git/info/sparse-checkout
    print("Cloning the 'datasets' folder...")
    !git pull origin main
    print("\nVerification:")
    !ls datasets
    print("\nDatasets downloaded successfully. Setting DATASETS_PATH to '/content/datasets/'")
    DATASETS_PATH = '/content/datasets/'
else:
    print("Not running in Google Colab. Assuming datasets are already available locally.")
    DATASETS_PATH = './datasets/'

Running in Google Colab. Downloading datasets...
Reinitialized existing Git repository in /content/.git/
error: remote origin already exists.
Cloning the 'datasets' folder...
From https://github.com/FrostPrice/kdd-queimadas-nas-ucs
 * branch            main       -> FETCH_HEAD
Already up to date.

Verification:
cnuc_2025_03.xlsx
planilha-de-area-atingida-por-fogo-historico-dgeo-3.xlsx
queimadas-brasil
relatorio_categorizacao_2019-portal.xls

Datasets downloaded successfully. Setting DATASETS_PATH to '/content/datasets/'


# Pré-Processamento

## Funções e Mappers

In [49]:
def map_values(df, col, map):
    for regex, new_value in map.items():
      df[col] = df[col].str.replace(regex, new_value, regex=True)

def normalize_strings(df, cols):
    for col in cols:
      df[col] = df[col].str.lower().str.replace(r'[^\w\s]', '', regex=True).str.replace(r'\s+', ' ', regex=True).apply(unidecode)

siglas_map = {
    r'^area de protecao ambiental': 'apa',
    r'^area de relevante interesse ecologico': 'arie',
    r'^estacao ecologica': 'esec',
    r'^floresta nacional': 'flona',
    r'^parque nacional': 'parna',
    r'^reserva biologica': 'rebio',
    r'^reserva extrativista': 'resex',
    r'^(refugio de vida silvestre|rvs)': 'revis',
    r'^(monumento natural|mona)': 'mn'
}

## Importação dos datasets

In [50]:
# Municípios Turísticos
municipios_df = pd.read_excel(f'{DATASETS_PATH}/relatorio_categorizacao_2019-portal.xls', usecols=[
    'MACRO',
    'UF',
    'REGIAO_TURISTICA',
    'MUNICIPIO',
    # 'COD_IBGE',
    'QUANTIDADE_EMPREGOS',
    'QUANTIDADE_ ESTABELECIMENTOS',
    'QUANTIDADE_VISITAS_ESTIMADAS_ INTERNACIONAL',
    'QUANTIDADE_VISITAS_ESTIMADAS_ NACIONAL',
    'ARRECADACAO',
    # 'CLUSTER'
])

municipios_df.rename(
    columns={
        'QUANTIDADE_ ESTABELECIMENTOS': 'QUANTIDADE_ESTABELECIMENTOS',
        'QUANTIDADE_VISITAS_ESTIMADAS_ INTERNACIONAL': 'QUANTIDADE_VISITAS_ESTIMADAS_INTERNACIONAL',
        'QUANTIDADE_VISITAS_ESTIMADAS_ NACIONAL': 'QUANTIDADE_VISITAS_ESTIMADAS_NACIONAL'
    },
    inplace=True
)
normalize_strings(municipios_df, ['MACRO', 'REGIAO_TURISTICA', 'MUNICIPIO'])

print(municipios_df.head())

         MACRO  UF                        REGIAO_TURISTICA   MUNICIPIO  \
0  centrooeste  MT  regiao turistica portal do agronegocio  nova mutum   
1     nordeste  MA            polo lagos e campos floridos     cajapio   
2      sudeste  MG                 caminhos da mantiqueira  brazopolis   
3      sudeste  MG                                    ouro       caete   
4      sudeste  MG                        pedras preciosas      catuji   

   QUANTIDADE_EMPREGOS  QUANTIDADE_ESTABELECIMENTOS  \
0                   89                           14   
1                    0                            0   
2                    6                            3   
3                  310                           11   
4                    0                            0   

   QUANTIDADE_VISITAS_ESTIMADAS_INTERNACIONAL  \
0                                           0   
1                                           0   
2                                           0   
3                           

In [59]:
# Unidades de Conservação
cnuc_df = pd.read_excel(
    f'{DATASETS_PATH}cnuc_2025_03.xlsx',
    usecols=[
        'Nome da UC',
        # 'Esfera Administrativa',
        'Categoria de Manejo',
        'Ano de Criação',
        'Municípios Abrangidos',
        # 'Bioma declarado',
        'Grupo'
    ]
)

cnuc_df.rename(columns={'Nome da UC': 'UC'}, inplace=True)
normalize_strings(cnuc_df, ['UC', 'Categoria de Manejo', 'Grupo'])
map_values(cnuc_df, 'UC', siglas_map)

print(cnuc_df.head())

                                             UC         Categoria de Manejo  \
0                               apa anhatomirim  area de protecao ambiental   
1                       apa cavernas do peruacu  area de protecao ambiental   
2                apa da bacia do rio descoberto  area de protecao ambiental   
3            apa da bacia do rio sao bartolomeu  area de protecao ambiental   
4  apa da bacia do rio sao joao micoleaodourado  area de protecao ambiental   

   Ano de Criação                              Municípios Abrangidos  \
0            1992                        GOVERNADOR CELSO RAMOS (SC)   
1            1989  BONITO DE MINAS (MG) - CÔNEGO MARINHO (MG) - I...   
2            1983  ÁGUAS LINDAS DE GOIÁS (GO) - BRASÍLIA (DF) - P...   
3            1983                                      BRASÍLIA (DF)   
4            2002  ARARUAMA (RJ) - CABO FRIO (RJ) - CACHOEIRAS DE...   

             Grupo  
0  uso sustentavel  
1  uso sustentavel  
2  uso sustentavel  
3  uso s

In [41]:
# Queimadas em UCs Federais
# 2015
queimadas_uc_2015 = pd.read_excel(f'{DATASETS_PATH}planilha-de-area-atingida-por-fogo-historico-dgeo-3.xlsx', sheet_name='2015', usecols=[
    'UC',
    'Área UC (ha)',
    'Área Queimada (ha)'
])

queimadas_uc_2015.rename(columns={'Área Queimada (ha)': '2015'}, inplace=True)
normalize_strings(queimadas_uc_2015, 'UC')

# 2016
queimadas_uc_2016 = pd.read_excel(f'{DATASETS_PATH}planilha-de-area-atingida-por-fogo-historico-dgeo-3.xlsx', sheet_name='2016', usecols=[
    'Unidades de Conservação',
    'Área UC (ha)',
    'Área Queimada (ha)'
])

queimadas_uc_2016.rename(columns={'Unidades de Conservação': 'UC', 'Área Queimada (ha)': '2016'}, inplace=True)
normalize_strings(queimadas_uc_2016, 'UC')

# 2017
queimadas_uc_2017 = pd.read_excel(f'{DATASETS_PATH}planilha-de-area-atingida-por-fogo-historico-dgeo-3.xlsx', sheet_name='2017', usecols=[
    'Unidades de Conservação',
    'Área UC (ha)',
    'Área Queimada (ha)'
])

queimadas_uc_2017.rename(columns={'Unidades de Conservação': 'UC', 'Área Queimada (ha)': '2017'}, inplace=True)
normalize_strings(queimadas_uc_2017, 'UC')

# 2018
queimadas_uc_2018 = pd.read_excel(f'{DATASETS_PATH}planilha-de-area-atingida-por-fogo-historico-dgeo-3.xlsx', sheet_name='2018', usecols=[
    'Unidades de Conservação',
    'Área UC (ha)',
    'Área Queimada (ha)'
])

queimadas_uc_2018.rename(columns={'Unidades de Conservação': 'UC', 'Área Queimada (ha)': '2018'}, inplace=True)
normalize_strings(queimadas_uc_2018, 'UC')

# 2019
queimadas_uc_2019 = pd.read_excel(f'{DATASETS_PATH}planilha-de-area-atingida-por-fogo-historico-dgeo-3.xlsx', sheet_name='2019', usecols=[
    'Unidades de Conservação',
    'Área UC (ha)',
    'Área Queimada (ha)'
])

queimadas_uc_2019.rename(columns={'Unidades de Conservação': 'UC', 'Área Queimada (ha)': '2019'}, inplace=True)
normalize_strings(queimadas_uc_2019, 'UC')

# 2020
queimadas_uc_2020 = pd.read_excel(f'{DATASETS_PATH}planilha-de-area-atingida-por-fogo-historico-dgeo-3.xlsx', sheet_name='2020', usecols=[
    'Unidades de Conservação',
    'Área UC (ha)',
    'Área Queimada (ha)'
])

queimadas_uc_2020.rename(columns={'Unidades de Conservação': 'UC', 'Área Queimada (ha)': '2020'}, inplace=True)
normalize_strings(queimadas_uc_2020, 'UC')

# 2021
queimadas_uc_2021 = pd.read_excel(f'{DATASETS_PATH}planilha-de-area-atingida-por-fogo-historico-dgeo-3.xlsx', sheet_name='2021', usecols=[
    'Unidades de Conservação',
    'Área UC (ha)',
    'Área Queimada (ha)'
])

queimadas_uc_2021.rename(columns={'Unidades de Conservação': 'UC', 'Área Queimada (ha)': '2021'}, inplace=True)
normalize_strings(queimadas_uc_2021, 'UC')

# 2022
queimadas_uc_2022 = pd.read_excel(f'{DATASETS_PATH}planilha-de-area-atingida-por-fogo-historico-dgeo-3.xlsx', sheet_name='2022', usecols=[
    'Unidade de Conservação ',
    'Área UC (ha) ',
    'Área total (ha) '
])

queimadas_uc_2022.rename(
    columns={
        'Unidade de Conservação ': 'UC',
        'Área UC (ha) ': 'Área UC (ha)',
        'Área total (ha) ': '2022'
    },
    inplace=True
)
normalize_strings(queimadas_uc_2022, 'UC')

print(queimadas_uc_2022.head())

                                            UC         2022  Área UC (ha)
0                      apa cavernas do peruacu   640.727481   143355.5780
1               apa da bacia do rio descoberto  2250.455588    41783.6133
2           apa da bacia do rio paraiba do sul   647.041726   292599.9380
3           apa da bacia do rio sao bartolomeu   557.247840    82680.8047
4  apa da bacia do rio sao joaomicoleaodourado   148.563685   150374.6090


In [54]:
queimadas_uc = [
    queimadas_uc_2015,
    queimadas_uc_2016,
    queimadas_uc_2017,
    queimadas_uc_2018,
    queimadas_uc_2019,
    queimadas_uc_2020,
    queimadas_uc_2021,
    queimadas_uc_2022
]

anos_frames = []
areas_frames = []

for df in queimadas_uc:
    map_values(df, 'UC', siglas_map)
    df = df.copy()
    # Detecta a coluna do ano (ex.: "2015")
    ano_cols = [c for c in df.columns if re.fullmatch(r'\d{4}', str(c))]
    if len(ano_cols) != 1:
        raise ValueError(f'Esperava 1 coluna de ano em {df.columns}, achei {ano_cols}')
    ano = ano_cols[0]

    # Garante 1 linha por UC (se houver duplicatas, leva o primeiro valor)
    df = df.groupby('UC', as_index=False).agg({'Área UC (ha)': 'first', ano: 'first'})

    anos_frames.append(df.set_index('UC')[[ano]])
    areas_frames.append(df.set_index('UC')[['Área UC (ha)']])

# Junta todos os anos lado a lado (outer join pelo índice UC)
wide = pd.concat(anos_frames, axis=1)

# Escolhe a área: primeiro valor não nulo entre os DFs
area_unificada = pd.concat(areas_frames, axis=1).bfill(axis=1).iloc[:, 0]
area_unificada.name = 'Área UC (ha)'

# Monta resultado final: UC, Área, anos (ordenados)
out = pd.concat([area_unificada, wide], axis=1).reset_index()
anos_ordenados = sorted([c for c in out.columns if re.fullmatch(r'\d{4}', str(c))])
queimadas_uc_df = out[['UC', 'Área UC (ha)'] + anos_ordenados]

display(queimadas_uc_df)

,UC,Área UC (ha),2015,2016,2017,2018,2019,2020,2021,2022
0,apa morro da pedreira,1.001313e+05,6668.971265,5852.609461,360.288892,NaN,NaN,NaN,NaN,4977.367832
1,esec da terra do meio,3.373066e+06,2599.194246,347.818726,8815.147000,365.316263,8201.561971,8466.720824,8117.579044,2941.719043
2,esec de caracarai,8.679498e+04,3975.506384,731.392841,NaN,NaN,NaN,NaN,NaN,NaN
3,esec de urucuiuna,1.371480e+05,59146.660148,28439.700000,27425.185562,19132.680000,39195.458278,43551.230920,NaN,40106.754174
4,esec raso da catarina,1.048444e+05,1758.154730,NaN,NaN,NaN,NaN,NaN,556.680000,NaN
...,...,...,...,...,...,...,...,...,...,...
151,rebio do tingua,2.481290e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.453923
152,resex da mata grande,1.143166e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,207.537688
153,resex do alto jurua,5.379494e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.851754
154,resex ituxi,7.763296e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.867540


## Filtros nos datasets

In [55]:
# Municípios Turísticos

# Remove tuplas com dados inválidos ou desnecessários para a análise
municipios_df = municipios_df[
    (municipios_df['QUANTIDADE_ESTABELECIMENTOS'] > 0)
    & (
        (municipios_df['QUANTIDADE_VISITAS_ESTIMADAS_NACIONAL'] > 0)
        | (municipios_df['QUANTIDADE_VISITAS_ESTIMADAS_INTERNACIONAL'] > 0)
    )
]

print(municipios_df.head())

         MACRO  UF                        REGIAO_TURISTICA  \
0  centrooeste  MT  regiao turistica portal do agronegocio   
2      sudeste  MG                 caminhos da mantiqueira   
3      sudeste  MG                                    ouro   
7      sudeste  SP      regiao turistica historias e vales   
8      sudeste  SP               entre rios serras e cafes   

               MUNICIPIO  QUANTIDADE_EMPREGOS  QUANTIDADE_ESTABELECIMENTOS  \
0             nova mutum                   89                           14   
2             brazopolis                    6                            3   
3                  caete                  310                           11   
7           pirassununga                   62                            7   
8  sao joao da boa vista                   62                           10   

   QUANTIDADE_VISITAS_ESTIMADAS_INTERNACIONAL  \
0                                           0   
2                                           0   
3          

In [60]:
# CNUC
# Remove UCs criadas após 2019
cnuc_df = cnuc_df[cnuc_df['Ano de Criação'] <= 2019]

# 1. Separar cidades pelo delimitador " - "
cnuc_df["Município"] = cnuc_df["Municípios Abrangidos"].str.split(" - ")

# 2. Explodir em múltiplas linhas
cnuc_df = cnuc_df.explode("Município").reset_index(drop=True)

# 3. Extrair a UF (entre parênteses)
# cnuc_df["UF"] = cnuc_df["Município"].str.extract(r"\((.*?)\)")

# 4. Limpar o nome da cidade (remover o "(UF)")
cnuc_df["Município"] = cnuc_df["Município"].str.replace(r"\s*\(.*?\)", "", regex=True).str.strip()
normalize_strings(cnuc_df, ['Município'])

cnuc_df.drop(columns=['Municípios Abrangidos'], inplace=True)

após filtro do ano: 14135


### Junção dos Datasets

In [ ]:
# Merge
merged_df = pd.merge(uc_df, municipios_df, left_on='Município', right_on='MUNICIPIO', how='left')
display(merged_df)

,Categoria de Manejo,Ano de Criação,Bioma declarado,Grupo,Município,UF,MUNICIPIO,QUANTIDADE_EMPREGOS,QUANTIDADE_ ESTABELECIMENTOS,QUANTIDADE_VISITAS_ESTIMADAS_ INTERNACIONAL,QUANTIDADE_VISITAS_ESTIMADAS_ NACIONAL,ARRECADACAO
0,Área de Proteção Ambiental,1992,Área Marinha,Uso Sustentável,governador celso ramos,SC,governador celso ramos,179.0,5.0,20405.0,9793.0,0.0
1,Área de Proteção Ambiental,1989,Cerrado,Uso Sustentável,bonito de minas,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Área de Proteção Ambiental,1989,Cerrado,Uso Sustentável,conego marinho,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Área de Proteção Ambiental,1989,Cerrado,Uso Sustentável,itacarambi,MG,itacarambi,6.0,4.0,82.0,4567.0,0.0
4,Área de Proteção Ambiental,1989,Cerrado,Uso Sustentável,januaria,MG,januaria,27.0,6.0,304.0,45769.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
4429,Área de Relevante Interesse Ecológico,2005,Mata Atlântica,Uso Sustentável,olinda,PE,olinda,350.0,25.0,10489.0,173519.0,3309224.0
4430,Área de Proteção Ambiental,2014,Mata Atlântica,Uso Sustentável,olinda,PE,olinda,350.0,25.0,10489.0,173519.0,3309224.0
4431,Área de Relevante Interesse Ecológico,2005,Mata Atlântica,Uso Sustentável,olinda,PE,olinda,350.0,25.0,10489.0,173519.0,3309224.0
4432,Parque,2004,Mata Atlântica,Proteção Integral,guaratuba,PR,guaratuba,265.0,25.0,13870.0,300792.0,1241365.0


## Análise

In [ ]:
uc_municipios_df = merged_df[merged_df['UF'].notna()]
print(f'Total de UCs em municípios turísticos: {uc_municipios_df.size}')
display(uc_municipios_df)
print(f'Total de Municípios: {municipios_df.size}')

Total de UCs em municípios turísticos: 35436


,Categoria de Manejo,Ano de Criação,Bioma declarado,Grupo,Município,UF,MUNICIPIO,QUANTIDADE_EMPREGOS,QUANTIDADE_ ESTABELECIMENTOS,QUANTIDADE_VISITAS_ESTIMADAS_ INTERNACIONAL,QUANTIDADE_VISITAS_ESTIMADAS_ NACIONAL,ARRECADACAO
0,Área de Proteção Ambiental,1992,Área Marinha,Uso Sustentável,governador celso ramos,SC,governador celso ramos,179.0,5.0,20405.0,9793.0,0.0
3,Área de Proteção Ambiental,1989,Cerrado,Uso Sustentável,itacarambi,MG,itacarambi,6.0,4.0,82.0,4567.0,0.0
4,Área de Proteção Ambiental,1989,Cerrado,Uso Sustentável,januaria,MG,januaria,27.0,6.0,304.0,45769.0,0.0
6,Área de Proteção Ambiental,1983,Cerrado,Uso Sustentável,brasilia,DF,brasilia,3562.0,220.0,112474.0,4332053.0,59764674.0
7,Área de Proteção Ambiental,1983,Cerrado,Uso Sustentável,padre bernardo,GO,padre bernardo,11.0,5.0,0.0,19606.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
4429,Área de Relevante Interesse Ecológico,2005,Mata Atlântica,Uso Sustentável,olinda,PE,olinda,350.0,25.0,10489.0,173519.0,3309224.0
4430,Área de Proteção Ambiental,2014,Mata Atlântica,Uso Sustentável,olinda,PE,olinda,350.0,25.0,10489.0,173519.0,3309224.0
4431,Área de Relevante Interesse Ecológico,2005,Mata Atlântica,Uso Sustentável,olinda,PE,olinda,350.0,25.0,10489.0,173519.0,3309224.0
4432,Parque,2004,Mata Atlântica,Proteção Integral,guaratuba,PR,guaratuba,265.0,25.0,13870.0,300792.0,1241365.0


Total de Municípios: 11193
